In [4]:
import os
import pandas as pd

ROOT_DIR = "../FMP"  # Cambia qui se la cartella principale ha un altro nome


In [5]:

def process_events_csv(filepath):
    try:
        df = pd.read_csv(filepath)
        if "Evento" not in df.columns or "Casella di gruppo" not in df.columns:
            print(f"⚠️  Colonne mancanti in {filepath}. Skippato.")
            return 0, False

        # Quante righe sistemiamo?
        mask = df["Evento"].isna() & df["Casella di gruppo"].notna()
        n_fixed = mask.sum()

        if n_fixed > 0:
            df.loc[mask, "Evento"] = df.loc[mask, "Casella di gruppo"]

        # Elimina la colonna
        if "Casella di gruppo" in df.columns:
            df = df.drop(columns=["Casella di gruppo"])

        # Sovrascrivi il file solo se abbiamo fatto modifiche
        if n_fixed > 0 or "Casella di gruppo" in df.columns:
            df.to_csv(filepath, index=False)
            return n_fixed, True

        return 0, False

    except Exception as e:
        print(f"❌ Errore su {filepath}: {e}")
        return 0, False

def scan_and_process(root_dir):
    report = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file == "events.csv":
                filepath = os.path.join(subdir, file)
                n_fixed, modified = process_events_csv(filepath)
                report.append((filepath, n_fixed, modified))
    return report



In [6]:
report = scan_and_process(ROOT_DIR)
print("\n=== REPORT FINALE ===")
for filepath, n_fixed, modified in report:
    status = "Aggiornato" if modified else "OK (nessuna modifica)"
    print(f"{filepath} -> {status} | Eventi sistemati: {n_fixed}")

⚠️  Colonne mancanti in FMP/Coppa/Jesolo/events.csv. Skippato.
⚠️  Colonne mancanti in FMP/Coppa/Conegliano/events.csv. Skippato.
⚠️  Colonne mancanti in FMP/Coppa/Miti/events.csv. Skippato.
⚠️  Colonne mancanti in FMP/Campionato/Jesolo/events.csv. Skippato.

=== REPORT FINALE ===
FMP/Coppa/Jesolo/events.csv -> OK (nessuna modifica) | Eventi sistemati: 0
FMP/Coppa/Conegliano/events.csv -> OK (nessuna modifica) | Eventi sistemati: 0
FMP/Coppa/Miti/events.csv -> OK (nessuna modifica) | Eventi sistemati: 0
FMP/Campionato/Jesolo/events.csv -> OK (nessuna modifica) | Eventi sistemati: 0
